In [224]:
import numpy as np
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

In [225]:
alldata= pd.read_csv("C:\\Users\\DELL\\Desktop\\spambase.data",header=None, names=list(range(0,58)))
alldata.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [231]:
X=alldata.iloc[:,:48].values
y=alldata.iloc[:,57].values

In [232]:
def classify(x_vector):
    # change percentage into qualitative feature(0 or 1)
    x_binary=[]
    for percent in x_vector:
        if percent>0.0:
            x_binary+=[1]
        else:
            x_binary+=[0]
    #  calculate_log_likelihoods_with_naive_bayes
    pos_log_likelihood=0.0
    neg_log_likelihood=0.0
    # for the qualitative features, raplace the f(pos)(x1,x,2,....xi) with calcutating probability mass function over 0 and 1 categories
    for idx in range(len(x_binary)):
        # feature present: P(X), taking log
        if x_binary[idx] == 1: 
            pos_log_likelihood += np.log10(likelihoods_pos[idx]) 
        #feature absent: 1-P(X), taking log
        elif x_binary[idx] == 0: 
            pos_log_likelihood += np.log10(1.0 - likelihoods_pos[idx])
    for idx in range(len(x_binary)):
        if x_binary[idx] == 1:
            neg_log_likelihood += np.log10(likelihoods_neg[idx]) 
        elif x_binary[idx] == 0: 
            neg_log_likelihood += np.log10(1.0 - likelihoods_neg[idx])
    # calculate posteriors
    # classify a new data according to a higher posterior Pr(Y=y|X=x)
    log_posterior_pos = pos_log_likelihood + log_prior_pos
    log_posterior_neg = neg_log_likelihood + log_prior_neg
    if log_posterior_neg > log_posterior_pos:
        return 0
    else:
        return 1

In [233]:
# Use 5-fold cross-validation 
skf =  StratifiedKFold(n_splits=5,random_state=123)

In [234]:
# varaiables to store test error in each fold
fp_rate,fn_rate,error=[],[],[]
# 5-fold cross validation to split training and test data
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train_pos=[]
    X_train_neg=[]
    # split the two classes in the training data
    for i in range(len(X_train)):
        if y_train[i]==1:
            X_train_pos+=[X_train[i]]
        else:
            X_train_neg+=[X_train[i]]
            
    # given the specific class(pos or neg),estimate the likelihoods of each featrures from training data
    likelihoods_pos = np.mean(X_train_pos, axis=0)/100.0
    likelihoods_neg = np.mean(X_train_neg, axis=0)/100.0
    
    #Calculate the class priors
    num_pos = len(X_train_pos)
    num_neg = len(X_train_neg)
    prior_probability_pos = num_pos / float(num_pos + num_neg)
    prior_probability_neg = num_neg / float(num_pos + num_neg)
    
    # Taking logs
    log_prior_pos = np.log10(prior_probability_pos)
    log_prior_neg = np.log10(prior_probability_neg)
    
    # for each test data,do the classification
    predictions = []
    for i in X_test:
        # classify_spam is the main fuction to do prediction and output the result
        predictions.append(classify(i))
        
    # calculate the overall test error
    error.append(sum(predictions!=y_test)/len(y_test))
    # using confusion_matrix to get TN,FP,FN,TP
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    # calculate the false postive rate and false negative rate
    fp_rate.append(fp/(fp+tn))
    fn_rate.append(fn/(fn+tp))

In [235]:
from prettytable import PrettyTable 
table=PrettyTable(["fold","false positive rate","false negative rate","overall error rate"])
for i in range(5):
    table.add_row([i+1,fp_rate[i],fn_rate[i],error[i]])
print(table)

+------+---------------------+---------------------+---------------------+
| fold | false positive rate | false negative rate |  overall error rate |
+------+---------------------+---------------------+---------------------+
|  1   |  0.0913978494623656 | 0.05509641873278237 | 0.07709011943539631 |
|  2   | 0.11827956989247312 | 0.03581267217630854 | 0.08577633007600434 |
|  3   |  0.1935483870967742 | 0.03305785123966942 | 0.13029315960912052 |
|  4   |  0.1059245960502693 | 0.04696132596685083 | 0.08269858541893363 |
|  5   | 0.39497307001795334 |  0.1132596685082873 |  0.2840043525571273 |
+------+---------------------+---------------------+---------------------+
